In [1]:
from pyspark.sql import SparkSession

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.max_columns = 100

import warnings
warnings.filterwarnings("ignore")

In [2]:
spark = SparkSession \
    .builder \
    .appName("_Practical_Spark_Ex") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
!pwd

/home/jovyan/work


In [4]:
!ls

kaggle_wind_analysis.ipynb	 solar_generation_by_station.csv
solar_generation_by_country.csv


In [7]:
!head solar_generation_by_country.csv

AT,BE,BG,CH,CY,CZ,DE,DK,EE,ES,FI,FR,EL,HR,HU,IE,IT,LT,LU,LV,NL,NO,PL,PT,RO,SI,SK,SE,UK
0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00544544157850945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0524502739386129,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0582243368779087,0.0141948060502403,

Wind Hourly Power Capacity

The datasets contain hourly power generation data for NUTS 2 regions in Europe. The objective is to develop an algorithm for the prediction of the power generation using factors like economic status, renewable energy interest and the non-renewable power generation.

In [31]:
df_country = spark.read.csv("solar_generation_by_country.csv", header=True, sep=",");
df_station = spark.read.csv("solar_generation_by_station.csv", header=True, sep=",");

In [32]:
df_country.printSchema()

root
 |-- AT: string (nullable = true)
 |-- BE: string (nullable = true)
 |-- BG: string (nullable = true)
 |-- CH: string (nullable = true)
 |-- CY: string (nullable = true)
 |-- CZ: string (nullable = true)
 |-- DE: string (nullable = true)
 |-- DK: string (nullable = true)
 |-- EE: string (nullable = true)
 |-- ES: string (nullable = true)
 |-- FI: string (nullable = true)
 |-- FR: string (nullable = true)
 |-- EL: string (nullable = true)
 |-- HR: string (nullable = true)
 |-- HU: string (nullable = true)
 |-- IE: string (nullable = true)
 |-- IT: string (nullable = true)
 |-- LT: string (nullable = true)
 |-- LU: string (nullable = true)
 |-- LV: string (nullable = true)
 |-- NL: string (nullable = true)
 |-- NO: string (nullable = true)
 |-- PL: string (nullable = true)
 |-- PT: string (nullable = true)
 |-- RO: string (nullable = true)
 |-- SI: string (nullable = true)
 |-- SK: string (nullable = true)
 |-- SE: string (nullable = true)
 |-- UK: string (nullable = true)



In [33]:
df_station.dtypes

[('time_step', 'string'),
 ('AT11', 'string'),
 ('AT21', 'string'),
 ('AT12', 'string'),
 ('AT31', 'string'),
 ('AT32', 'string'),
 ('AT22', 'string'),
 ('AT33', 'string'),
 ('AT34', 'string'),
 ('AT13', 'string'),
 ('BE21', 'string'),
 ('BE31', 'string'),
 ('BE32', 'string'),
 ('BE33', 'string'),
 ('BE22', 'string'),
 ('BE34', 'string'),
 ('BE35', 'string'),
 ('BE23', 'string'),
 ('BE10', 'string'),
 ('BE24', 'string'),
 ('BE25', 'string'),
 ('BG32', 'string'),
 ('BG33', 'string'),
 ('BG31', 'string'),
 ('BG34', 'string'),
 ('BG41', 'string'),
 ('BG42', 'string'),
 ('CZ06', 'string'),
 ('CZ03', 'string'),
 ('CZ08', 'string'),
 ('CZ01', 'string'),
 ('CZ05', 'string'),
 ('CZ04', 'string'),
 ('CZ02', 'string'),
 ('CZ07', 'string'),
 ('DEA5', 'string'),
 ('DE30', 'string'),
 ('DE40', 'string'),
 ('DE91', 'string'),
 ('DE50', 'string'),
 ('DED1', 'string'),
 ('DE71', 'string'),
 ('DEE1', 'string'),
 ('DEA4', 'string'),
 ('DED2', 'string'),
 ('DEA1', 'string'),
 ('DE13', 'string'),
 ('DE72'

In [34]:
df_station.count()

262968

In [35]:
df_country.count()

262968

In [36]:
country_cols = df_country.schema.names

In [37]:
df_country.select(df_country.AT.cast("float").alias('AT'))

DataFrame[AT: float]

In [41]:
df_country.select('AT').cast('float')

AttributeError: 'DataFrame' object has no attribute 'cast'

In [40]:
df_country.dtypes

[('AT', 'string'),
 ('BE', 'string'),
 ('BG', 'string'),
 ('CH', 'string'),
 ('CY', 'string'),
 ('CZ', 'string'),
 ('DE', 'string'),
 ('DK', 'string'),
 ('EE', 'string'),
 ('ES', 'string'),
 ('FI', 'string'),
 ('FR', 'string'),
 ('EL', 'string'),
 ('HR', 'string'),
 ('HU', 'string'),
 ('IE', 'string'),
 ('IT', 'string'),
 ('LT', 'string'),
 ('LU', 'string'),
 ('LV', 'string'),
 ('NL', 'string'),
 ('NO', 'string'),
 ('PL', 'string'),
 ('PT', 'string'),
 ('RO', 'string'),
 ('SI', 'string'),
 ('SK', 'string'),
 ('SE', 'string'),
 ('UK', 'string')]

In [13]:
def pretty_summary(df):
    """ Neat summary statistics of a Spark dataframe
    Args: pyspark.sql.dataframe.DataFrame (df): input dataframe
    Returns:pandas.core.frame.DataFrame: a pandas dataframe with the summary statistics of df
    """
    import pandas as pd
    temp = df.describe().toPandas()
    temp.iloc[1:3,1:] = temp.iloc[1:3,1:].convert_objects(convert_numeric=True)
    pd.options.display.float_format = '{:,.2f}'.format
    return temp

In [14]:
pretty_summary(df_station)

,summary,time_step,AT11,AT21,AT12,AT31,AT32,AT22,AT33,AT34,AT13,BE21,BE31,BE32,BE33,BE22,BE34,BE35,BE23,BE10,BE24,BE25,BG32,BG33,BG31,BG34,BG41,BG42,CZ06,CZ03,CZ08,CZ01,CZ05,CZ04,CZ02,CZ07,DEA5,DE30,DE40,DE91,DE50,DED1,DE71,DEE1,DEA4,DED2,DEA1,DE13,DE72,DEE2,...,RO42,SE32,SE31,SE12,SE33,SE21,SE11,SE22,SE23,SK01,SK03,SK04,SK02,UKH2,UKJ1,UKD6,UKK3,UKD1,UKF1,UKK4,UKK2,UKH1,UKE1,UKL2,UKM2,UKH3,UKK1,UKD3,UKJ3,UKG1,UKM6,UKI3UKI4,UKJ4,UKD4,UKF2,UKF3,UKD7,UKM5,UKE2,UKN0,UKC2,UKI5UKI6,UKG2,UKM3,UKE3,UKJ2,UKC1,UKG3,UKL1,UKE4
0,count,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,...,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968,262968
1,mean,"131,484.50",0.14,0.14,0.13,0.12,0.12,0.13,0.12,0.12,0.13,0.12,0.12,0.12,0.11,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.15,0.15,0.15,0.15,0.15,0.16,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.11,0.12,0.12,0.11,0.11,0.11,0.12,0.12,0.11,0.12,0.12,0.13,0.12,0.12,...,0.14,0.09,0.10,0.11,0.00,0.11,0.11,0.11,0.10,0.13,0.13,0.13,0.14,0.12,0.12,0.11,0.12,0.10,0.11,0.12,0.12,0.12,0.11,0.11,0.10,0.12,0.12,0.10,0.12,0.11,0.09,0.12,0.12,0.10,0.11,0.11,0.11,0.10,0.10,0.10,0.10,0.12,0.11,0.10,0.11,0.12,0.10,0.11,0.11,0.10
2,stddev,"75,912.47",0.22,0.21,0.20,0.20,0.19,0.20,0.19,0.19,0.22,0.20,0.20,0.20,0.19,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.23,0.23,0.23,0.23,0.23,0.23,0.20,0.20,0.20,0.21,0.20,0.19,0.20,0.20,0.19,0.20,0.19,0.19,0.19,0.19,0.20,0.20,0.19,0.20,0.19,0.20,0.20,0.20,...,0.22,0.17,0.17,0.18,0.00,0.18,0.20,0.19,0.18,0.22,0.20,0.21,0.22,0.19,0.19,0.18,0.20,0.17,0.18,0.19,0.19,0.19,0.18,0.18,0.16,0.20,0.19,0.17,0.20,0.18,0.15,0.20,0.20,0.17,0.19,0.19,0.19,0.16,0.17,0.16,0.17,0.19,0.18,0.17,0.18,0.20,0.17,0.18,0.18,0.17
3,min,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,max,99999,8e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,...,9e-05,9e-05,9e-05,9e-05,0,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,8e-05,9e-05,9e-05,9e-05,9e-05,9e-05,8e-05,8e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05,9e-05
